In [1]:
library(Matching)
library(twang)
library(weights)
library(tidyverse)

Loading required package: MASS
## 
##  Matching (Version 4.9-2, Build Date: 2015-12-25)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##

Loading required package: gbm
Loading required package: survival
Loading required package: lattice
Loading required package: splines
Loading required package: parallel
Loaded gbm 2.1.3
Loading required package: survey
Loading required package: grid
Loading required package: Matrix

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

Loading required package: xtable
Loading required package: latticeExtra
Loading required package: RColorBrewer
Loading required package: Hmisc
Loading required package: Formula
Loading required package: ggplot2

At

In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds")) %>%
    mutate(sunday = factor(as.integer(icu_adm_weekday == 0), levels = c(0, 1)),
           monday = factor(as.integer(icu_adm_weekday == 1), levels = c(0, 1)),
           tuesday = factor(as.integer(icu_adm_weekday == 2), levels = c(0, 1)),
           wednesday = factor(as.integer(icu_adm_weekday == 3), levels = c(0, 1)),
           thursday = factor(as.integer(icu_adm_weekday == 4), levels = c(0, 1)),
           friday = factor(as.integer(icu_adm_weekday == 5), levels = c(0, 1)),
           saturday = factor(as.integer(icu_adm_weekday == 6), levels = c(0, 1))) %>%
    mutate(female = factor(as.integer(as.integer(as.character(gender)) == 0), levels = c(0, 1)))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,ps,ps_weight,sunday,monday,tuesday,wednesday,thursday,friday,saturday,female
201220,125078,66690,0,2106-04-27 09:47:50,2106-05-01 19:25:46,1,62.67646,1,NA,⋯,0.4544669,1.833069,0,0,1,0,0,0,0,0
215842,151232,11663,0,2188-02-14 09:48:15,2188-02-16 03:02:48,1,86.76186,1,NA,⋯,0.1568838,1.186076,0,0,0,0,1,0,0,1
234312,164444,86645,1,2165-06-22 09:47:16,2165-07-07 21:55:20,1,56.08904,1,NA,⋯,0.3002571,1.429096,0,0,0,0,0,0,1,0
289157,146726,10304,0,2156-06-23 22:26:00,2156-06-30 17:26:00,1,45.91093,1,2156-06-25 08:00:00,⋯,0.6991412,1.430326,0,0,0,1,0,0,0,0
211964,160170,94534,0,2160-03-05 22:23:19,2160-03-07 06:48:41,1,59.38693,1,NA,⋯,0.3432285,1.522600,0,0,0,1,0,0,0,0
230173,112553,31544,0,2140-01-31 04:39:25,2140-02-03 02:41:39,1,91.50000,1,NA,⋯,0.2464983,1.327137,0,0,0,0,0,0,1,1


In [4]:
library(tableone)

In [5]:
tab_features <- CreateTableOne(vars = c("vs_map_first", "vs_heart_rate_first", "vs_temp_first", "vs_cvp_first",
                                        "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
                                        "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
                                        "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
                                        "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
                                        "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag"),
                               strata = "echo",
                               factorVars = c("lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag"),
                               data = full_data)
tab_features$ContTable

                                   Stratified by echo
                                    0               1               p      test
  n                                 3099            3063                       
  vs_map_first (mean (sd))           79.79 (19.25)   80.07 (19.95)   0.569     
  vs_heart_rate_first (mean (sd))    93.01 (19.81)   94.96 (21.76)  <0.001     
  vs_temp_first (mean (sd))          36.75 (1.05)    36.83 (1.58)    0.013     
  vs_cvp_first (mean (sd))           11.88 (17.09)   13.57 (19.12)   0.034     
  lab_wbc_first (mean (sd))          13.48 (14.03)   13.91 (12.58)   0.215     
  lab_hemoglobin_first (mean (sd))   10.55 (1.98)    10.65 (2.04)    0.062     
  lab_platelet_first (mean (sd))    223.96 (134.79) 213.44 (127.53)  0.002     
  lab_sodium_first (mean (sd))      139.09 (6.45)   138.62 (5.92)    0.003     
  lab_potassium_first (mean (sd))     4.10 (0.79)     4.15 (0.85)    0.024     
  lab_bicarbonate_first (mean (sd))  22.28 (5.40)    22.26 (5.71) 

In [6]:
tab_basics <- CreateTableOne(vars = c("age", "gender", "weight", "elix_score",
                                      "saps", "sofa", "vent", "vaso", "sedative",
                                      "icd_chf", "icd_afib", "icd_renal", "icd_liver",
                                      "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
                                      "icu_adm_weekday"),
                             strata = "echo",
                             factorVars = c("gender", "vent", "vaso", "sedative",
                                            "icd_chf", "icd_afib", "icd_renal", "icd_liver",
                                            "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
                                            "icu_adm_weekday"),
                             data = full_data)

In [7]:
print(tab_basics, smd = TRUE)

                        Stratified by echo
                         0             1             p      test SMD   
  n                       3099          3063                           
  age (mean (sd))        66.69 (17.21) 65.82 (16.62)  0.045       0.051
  gender = 1 (%)          1524 (49.2)   1603 (52.3)   0.014       0.063
  weight (mean (sd))     78.56 (23.58) 82.98 (26.70) <0.001       0.175
  elix_score (mean (sd))  8.51 (7.45)  10.05 (7.68)  <0.001       0.203
  saps (mean (sd))       19.63 (5.79)  20.78 (5.45)  <0.001       0.204
  sofa (mean (sd))        5.30 (3.62)   6.32 (3.80)  <0.001       0.275
  vent = 1 (%)            1459 (47.1)   1808 (59.0)  <0.001       0.241
  vaso = 1 (%)             839 (27.1)   1176 (38.4)  <0.001       0.243
  sedative = 1 (%)        1247 (40.2)   1540 (50.3)  <0.001       0.203
  icd_chf = 1 (%)          563 (18.2)   1198 (39.1)  <0.001       0.476
  icd_afib = 1 (%)         622 (20.1)    988 (32.3)  <0.001       0.280
  icd_renal = 1 (%)  

In [8]:
vars_ <- c("age", "female", "weight", "elix_score",
           "saps", "sofa", "vent", "vaso", "sedative",
           "icd_chf", "icd_afib", "icd_renal", "icd_liver",
           "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
           "sunday", "monday", "tuesday", "wednesday",
           "thursday", "friday", "saturday",
           "vs_map_first", "vs_heart_rate_first", "vs_temp_first", "vs_cvp_first",
           "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
           "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
           "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
           "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
           "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag")

factor_vars_ <- c("gender", "vent", "vaso", "sedative",
                  "icd_chf", "icd_afib", "icd_renal", "icd_liver",
                  "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
                  "sunday", "monday", "tuesday", "wednesday",
                  "thursday", "friday", "saturday",
                  "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag")

In [9]:
tabs <- CreateTableOne(vars = vars_, factorVars = factor_vars_,
                       strata = "echo", data = full_data,
                       argsNormal = list(var.equal = FALSE))

In [10]:
tabs_df <- print(tabs, smd = TRUE) %>% invisible %>% as.data.frame(stringsAsFactors = FALSE)

                                    Stratified by echo
                                     0               1               p     
  n                                    3099            3063                
  age (mean (sd))                     66.69 (17.21)   65.82 (16.62)   0.045
  female = 1 (%)                       1575 (50.8)     1460 (47.7)    0.014
  weight (mean (sd))                  78.56 (23.58)   82.98 (26.70)  <0.001
  elix_score (mean (sd))               8.51 (7.45)    10.05 (7.68)   <0.001
  saps (mean (sd))                    19.63 (5.79)    20.78 (5.45)   <0.001
  sofa (mean (sd))                     5.30 (3.62)     6.32 (3.80)   <0.001
  vent = 1 (%)                         1459 (47.1)     1808 (59.0)   <0.001
  vaso = 1 (%)                          839 (27.1)     1176 (38.4)   <0.001
  sedative = 1 (%)                     1247 (40.2)     1540 (50.3)   <0.001
  icd_chf = 1 (%)                       563 (18.2)     1198 (39.1)   <0.001
  icd_afib = 1 (%)               

In [11]:
tabs_df %>% head

,0,1,p,test,SMD
n,3099,3063,,,
age (mean (sd)),66.69 (17.21),65.82 (16.62),0.045,,0.051
female = 1 (%),1575 (50.8),1460 (47.7),0.014,,0.063
weight (mean (sd)),78.56 (23.58),82.98 (26.70),<0.001,,0.175
elix_score (mean (sd)),8.51 (7.45),10.05 (7.68),<0.001,,0.203
saps (mean (sd)),19.63 (5.79),20.78 (5.45),<0.001,,0.204


In [12]:
tabs_df_fmt <- tabs_df %>%
setNames(c("control", "treated", "pval", "test", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate(cov_name = str_split(cov, " ") %>% map_chr(first)) %>%
mutate(cate = cov_name %in% factor_vars_) %>%
mutate(ctrl = ifelse(cate,
                     str_extract(control, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     control)) %>%
mutate(trtd = ifelse(cate,
                     str_extract(treated, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     treated)) %>%
mutate(cov_fmt = str_replace_all(cov_name, "_", " ")) %>%
mutate(cov_fmt = str_replace_all(cov_fmt, "lab|vs|flag|icd|first", "")) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov_fmt = tools::toTitleCase(cov_fmt)) %>%
select(cov_fmt, ctrl, trtd, pval, smd) %>%
setNames(c("Covariate", "Non-Echo", "Echo", "p value", "SMD"))

In [13]:
tabs_df_fmt

Covariate,Non-Echo,Echo,p value,SMD
n,3099,3063,,
Age,66.69 (17.21),65.82 (16.62),0.045,0.051
Female,1575 (50.8),1460 (47.7),0.014,0.063
Weight,78.56 (23.58),82.98 (26.70),<0.001,0.175
Elix Score,8.51 (7.45),10.05 (7.68),<0.001,0.203
Saps,19.63 (5.79),20.78 (5.45),<0.001,0.204
Sofa,5.30 (3.62),6.32 (3.80),<0.001,0.275
Vent,47.1%,59.0%,<0.001,0.241
Vaso,27.1%,38.4%,<0.001,0.243
Sedative,40.2%,50.3%,<0.001,0.203


In [14]:
set.seed(4958)

In [15]:
ps_matches <- Match(Y = NULL, Tr = full_data$echo_int, X = full_data$ps, M = 1,
                    estimand = "ATT", caliper = 0.01,
                    exact = FALSE, replace = FALSE)

In [16]:
tab <- table(full_data$mort_28_day[ps_matches$index.treated],
             full_data$mort_28_day[ps_matches$index.control],
             dnn = c("Echo", "Control"))
tab

    Control
Echo   0   1
   0 817 349
   1 261 123

In [17]:
tabs_ps <- CreateTableOne(vars = vars_, factorVars = factor_vars_,
                          strata = "echo", argsNormal = list(var.equal = FALSE),
                          data = full_data[unlist(ps_matches[c("index.treated", "index.control")]), ])

In [18]:
tabs_ps_df <- print(tabs_ps, smd = TRUE) %>% as.data.frame(stringsAsFactors = FALSE)

                                    Stratified by echo
                                     0               1               p     
  n                                    1550            1550                
  age (mean (sd))                     66.59 (16.60)   66.67 (16.73)   0.891
  female = 1 (%)                        735 (47.4)      758 (48.9)    0.429
  weight (mean (sd))                  81.90 (25.01)   79.19 (22.91)   0.003
  elix_score (mean (sd))               9.41 (7.59)     8.69 (7.45)    0.008
  saps (mean (sd))                    20.27 (5.38)    20.03 (5.41)    0.217
  sofa (mean (sd))                     6.02 (3.70)     5.55 (3.58)   <0.001
  vent = 1 (%)                          827 (53.4)      802 (51.7)    0.388
  vaso = 1 (%)                          518 (33.4)      491 (31.7)    0.319
  sedative = 1 (%)                      705 (45.5)      690 (44.5)    0.613
  icd_chf = 1 (%)                       438 (28.3)      335 (21.6)   <0.001
  icd_afib = 1 (%)               

In [19]:
tabs_ps_df %>% head

,0,1,p,test,SMD
n,1550,1550,,,
age (mean (sd)),66.59 (16.60),66.67 (16.73),0.891,,0.005
female = 1 (%),735 (47.4),758 (48.9),0.429,,0.030
weight (mean (sd)),81.90 (25.01),79.19 (22.91),0.003,,0.113
elix_score (mean (sd)),9.41 (7.59),8.69 (7.45),0.008,,0.095
saps (mean (sd)),20.27 (5.38),20.03 (5.41),0.217,,0.044


In [20]:
tabs_ps_df_fmt <- tabs_ps_df %>%
setNames(c("control", "treated", "pval", "test", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate(cov_name = str_split(cov, " ") %>% map_chr(first)) %>%
mutate(cate = cov_name %in% factor_vars_) %>%
mutate(ctrl = ifelse(cate,
                     str_extract(control, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     control)) %>%
mutate(trtd = ifelse(cate,
                     str_extract(treated, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     treated)) %>%
mutate(cov_fmt = str_replace_all(cov_name, "_", " ")) %>%
mutate(cov_fmt = str_replace_all(cov_fmt, "lab|vs|flag|icd|first", "")) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov_fmt = tools::toTitleCase(cov_fmt)) %>%
select(cov_fmt, ctrl, trtd, pval, smd) %>%
setNames(c("Covariate", "Non-Echo (PS matching)", "Echo (PS matching)", "p value (PS matching)", "SMD (PS matching)"))

In [21]:
tabs_ps_df_fmt

Covariate,Non-Echo (PS matching),Echo (PS matching),p value (PS matching),SMD (PS matching)
n,1550,1550,,
Age,66.59 (16.60),66.67 (16.73),0.891,0.005
Female,735 (47.4),758 (48.9),0.429,0.030
Weight,81.90 (25.01),79.19 (22.91),0.003,0.113
Elix Score,9.41 (7.59),8.69 (7.45),0.008,0.095
Saps,20.27 (5.38),20.03 (5.41),0.217,0.044
Sofa,6.02 (3.70),5.55 (3.58),<0.001,0.129
Vent,53.4%,51.7%,0.388,0.032
Vaso,33.4%,31.7%,0.319,0.037
Sedative,45.5%,44.5%,0.613,0.019


In [22]:
data.table::fwrite(tabs_df_fmt, file.path(data_dir, "tableone_smd.csv"))
data.table::fwrite(tabs_ps_df_fmt, file.path(data_dir, "tableone_ps_smd.csv"))

In [23]:
tbone <- data.table::fread(file.path(data_dir, "tableone.csv"), data.table = FALSE)
tbone

Covariate,Echo,Non-Echo,p value,Echo (weighted cohort),Non-Echo (weighted cohort),p value (weighted cohort)
Age,65.82 (16.62),66.69 (17.21),0.045,66.06 (16.64),66.85 (16.79),0.064
Female,47.67%,50.82%,0.014,48.20%,49.39%,0.35
Weight,82.98 (26.70),78.56 (23.58),<0.001,81.25 (25.26),80.24 (24.31),0.128
Elix Score,10.05 (7.68),8.51 (7.45),<0.001,9.44 (7.64),9.10 (7.55),0.079
Saps,20.78 (5.45),19.63 (5.79),<0.001,20.23 (5.42),20.01 (5.65),0.12
Sofa,6.32 (3.80),5.30 (3.62),<0.001,5.86 (3.70),5.66 (3.66),0.031
Vent,59.03%,47.08%,<0.001,54.11%,50.67%,0.007
Vaso,38.39%,27.07%,<0.001,33.91%,30.84%,0.01
Sedative,50.28%,40.24%,<0.001,46.18%,43.09%,0.015
Chf,39.11%,18.17%,<0.001,30.38%,25.43%,<0.001


In [24]:
tabs_full <- tabs_df_fmt %>% select(Covariate, SMD) %>%
full_join(tbone, by = "Covariate") %>%
full_join(tabs_ps_df_fmt, by = "Covariate")

tabs_full

Covariate,SMD,Echo,Non-Echo,p value,Echo (weighted cohort),Non-Echo (weighted cohort),p value (weighted cohort),Non-Echo (PS matching),Echo (PS matching),p value (PS matching),SMD (PS matching)
n,,NA,NA,NA,NA,NA,NA,1550,1550,,
Age,0.051,65.82 (16.62),66.69 (17.21),0.045,66.06 (16.64),66.85 (16.79),0.064,66.59 (16.60),66.67 (16.73),0.891,0.005
Female,0.063,47.67%,50.82%,0.014,48.20%,49.39%,0.35,735 (47.4),758 (48.9),0.429,0.030
Weight,0.175,82.98 (26.70),78.56 (23.58),<0.001,81.25 (25.26),80.24 (24.31),0.128,81.90 (25.01),79.19 (22.91),0.003,0.113
Elix Score,0.203,10.05 (7.68),8.51 (7.45),<0.001,9.44 (7.64),9.10 (7.55),0.079,9.41 (7.59),8.69 (7.45),0.008,0.095
Saps,0.204,20.78 (5.45),19.63 (5.79),<0.001,20.23 (5.42),20.01 (5.65),0.12,20.27 (5.38),20.03 (5.41),0.217,0.044
Sofa,0.275,6.32 (3.80),5.30 (3.62),<0.001,5.86 (3.70),5.66 (3.66),0.031,6.02 (3.70),5.55 (3.58),<0.001,0.129
Vent,0.241,59.03%,47.08%,<0.001,54.11%,50.67%,0.007,53.4%,51.7%,0.388,0.032
Vaso,0.243,38.39%,27.07%,<0.001,33.91%,30.84%,0.01,33.4%,31.7%,0.319,0.037
Sedative,0.203,50.28%,40.24%,<0.001,46.18%,43.09%,0.015,45.5%,44.5%,0.613,0.019


In [25]:
data.table::fwrite(tabs_full, file.path(data_dir, "tableone_full.csv"))